In [1]:
#pip install faker

In [55]:
from faker import Faker
import csv
import uuid
import random
from decimal import Decimal
from datetime import datetime
from clickzetta.zettapark.session import Session
import json
RECORD_COUNT = 10000
fake = Faker()

In [56]:
current_time = datetime.now().strftime("%Y%m%d%H%M%S")
print(current_time)

20250109180445


In [57]:
file_path = f'FakeDataset/customer_{current_time}.csv'

In [58]:
def create_csv_file():
    with open(file_path, 'w', newline='') as csvfile:
        fieldnames = ["customer_id","first_name","last_name","email","street",
                      "city","state","country"
                     ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(RECORD_COUNT):
            #print(i)
            writer.writerow(
                {
                    "customer_id": str(uuid.uuid4()),
                    'first_name': fake.first_name(),
                    'last_name': fake.last_name(),
                    'email': fake.email(),
                    'street': fake.street_address(),
                    'city': fake.city(),
                    'state': fake.state(),
                    'country': fake.country()
                }
            )

In [59]:
def put_file_into_volume():
    # 从配置文件中读取参数
    with open('security/config-uat.json', 'r') as config_file:
        config = json.load(config_file)
    
    # 创建会话
    session = Session.builder.configs(config).create()
    session.file.put(file_path,"volume://scd_demo/")
    session.sql("show volume directory scd_demo").show()
    session.close()

In [60]:
if __name__ == '__main__':
    create_csv_file()
    put_file_into_volume()

--------------------------------------------------------------------------------------------------------------------------
|relative_path                |url                                                 |size     |last_modified_time         |
--------------------------------------------------------------------------------------------------------------------------
|customer_20250109180445.csv  |oss://czsampledatahz/scd_demo/customer_20250109...  |1323947  |2025-01-09 18:04:52+08:00  |
--------------------------------------------------------------------------------------------------------------------------

